In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

# Radio Detectability of Hot Jupiters
For instruments GMRT, HERA, LOFAR, SKA, UTR-2, VLA, we want to estimate Sensitivity, Time, Resolution, Confusion Limit

The radiometer equation:
$$
\frac{\sigma}{1 \rm~Jy} = 10^{23} \frac{2 k}{\lambda^2} \frac{T_{\rm sys} \Omega_{\rm station}}{\sqrt{2 B t N^2/2}}
$$

where $B$ is the bandwidth, $t$ is the observing time, and $N$ is the number of antennas.

or, since Omega ~ (lambda / D)^2:

$$
\frac{\sigma}{1 \rm~Jy} = 10^{23} \frac{2 k}{D^2} \frac{T_{\rm sys}}{\sqrt{2 B t N^2/2}}
$$




Galactic synch dominates below 100MHz (Di Matteo+ 2002, Santos, Cooray, Knox 2005, McQuinn+ 2006)

Tsys from Rogers & Bowman (2008)  
Tsys = Tsky = 237K(nu/150MHz)**-2.52

In [2]:
kB = 1.38e-16
degsq_to_steradians = (np.pi / 180.)**2
cc = 3.e10  # in cgs

def rms_Jy(observatory, tt, nu=70e6):
    """
    Returns the RMS noise in Jy for an observatory for a certain time and frequency.
    
    Default frequency is 70 MHz.
    """
    nn = observatory['num_antennas']
    Omega = np.pi * (observatory['angle_diam_deg']/2.)**2 * degsq_to_steradians  # in steradians
    Tsys = 238 * (nu / 150e6)**(-2.2)
    lam = cc / nu
    bb = observatory['bandwidth']
    prefactor = 1e23 * 2. * kB / lam**2
    return prefactor * Tsys * Omega / np.sqrt(2 * bb * tt * nn**2 / 2.)

In [17]:
# get table data
observatory_data = read_csv('telescope_data.csv')
observatory_data

,telescope,dish_diam_m,max_baseline_m,num_antennas
0,gmrt,45.00,25000,30
1,hera,14.00,1000,350
2,lofar_lba,35.00,100,44
3,mwa,4.00,1000,128
4,ska,1.75,65,130000
5,vla,25.00,36000,29
6,lwa,2.00,100,256


In [8]:
# We'll do the calculation assuming 70 MHz and 1 hr, with a bandwidth of 100 MHz
# that's a wavelength of 4.3 meters

In [5]:
observatories = {}
for ii, observatory in enumerate(observatory_data.telescope):
    this_observatory = {}
    for this_property in observatory_data:
        if this_property == 'telescope':
            continue
        this_observatory[this_property] = observatory_data[this_property].iloc[ii]
    observatories[observatory] = this_observatory
observatories

{'gmrt': {'dish_diam_m': 45.0, 'max_baseline_m': nan, 'num_antennas': 30.0},
 'hera': {'dish_diam_m': 14.0,
  'max_baseline_m': 1000.0,
  'num_antennas': 350.0},
 'lofar': {'dish_diam_m': 35.0, 'max_baseline_m': nan, 'num_antennas': nan},
 'lwa': {'dish_diam_m': 2.0, 'max_baseline_m': 100.0, 'num_antennas': 256.0},
 'mwa': {'dish_diam_m': 4.0, 'max_baseline_m': nan, 'num_antennas': 128.0},
 'ska': {'dish_diam_m': nan, 'max_baseline_m': nan, 'num_antennas': nan},
 'vla': {'dish_diam_m': 25.0, 'max_baseline_m': nan, 'num_antennas': 29.0}}

# Confusion noise
Want 1 source per 10 beams (pixels)  
Perley & Ericson (1974)  
Wieringa (1991)

$$
\sigma^2 = \int_\Omega \int_0^{5 \sigma / P} s^2 n[s] B[\Omega]^2 P[\Omega]^2 ds d\Omega
$$

$s$: source flux  
$n$: # count at $s$   -- $k s^{-1.65}$ (close to 3/2) ... and $k \propto \nu^{-0.7}$  
$B$: synthesized beam  
$P$: primary beam  
$ds$: sources  
$d\Omega$: solid angle

Remove all sources above 5$\sigma$

At 150 MHz with MWA, for $P \sim 30^\circ$, $B \sim 3.4'$, we get $\sigma \sim 0.1 \rm~Jy$.

$b_{\rm max}$ is the maximum baseline

Should scale as
$$
\sigma^2 \propto \left( \frac{\lambda}{D} \right)^2  \left( \frac{\lambda}{b_{\rm max}} \right)^2 \, ,
$$
or
$$
\sigma \propto \lambda^2 ,
$$

so at 70 MHz should be a factor of 4 worse.

So,

$$
5\sigma \sim 0.1 {\rm~Jy} \times \left( \frac{\nu}{150 \rm~MHz} \right)^{-2} \left( \frac{D}{4 \rm~m} \right)^{-2} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2}
$$

In the dense regime, the $D$ drops out.  In the sparse regime, the $D$ remains.

$$
\sigma \sim 0.2 {\rm~Jy} \left( \frac{\nu}{1 \rm~GHz} \right)^{0.7} \left( \frac{\theta}{1'} \right)^2
$$

TODO:  
replace $\theta$ with $\lambda / D$ and express in $\nu$ and $D$, and multiply by 5 for crude estimate of $5\sigma$.  
Wait, should it actually be $\lambda / b_{\rm max}$??  
I'm going to go with that.

\begin{eqnarray}
5 \sigma & \sim {\rm~Jy} \left( \frac{\nu}{1 \rm~GHz} \right)^{0.7} \left( \frac{\lambda / b_{\rm max}}{1'} \frac{60'}{1^\circ} \frac{180^\circ}{\pi \rm~radians} \right)^2 \\\
 & = 1.18 \times 10^7 {\rm~Jy} \left( \frac{\nu}{1 \rm~GHz} \right)^{0.7} \left( \frac{\lambda}{b_{\rm max}} \right)^2 \\\
 & = 1.18 \times 10^7 {\rm~Jy} \left( \frac{\nu}{1 \rm~GHz} \right)^{0.7} \left( \lambda \right)^2 \left( b_{\rm max} \right)^{-2} \\\
 & = 1.18 \times 10^7 {\rm~Jy} \left( \frac{\nu}{1 \rm~GHz} \right)^{0.7} \left( c / \nu \right)^2 \left( b_{\rm max} \right)^{-2} \\\
 & = 1.06 \times 10^{17} {\rm~Jy} \left( \frac{\nu}{1 \rm~GHz} \right)^{0.7} {\rm~m^2~s^{-2}} \left( \nu \right)^{-2} \left( D \right)^{-2} \\\
 & = 1.06 \times 10^{17} {\rm~Jy} \left( \frac{\nu}{1 \rm~GHz} \right)^{0.7} {\rm~m^2~s^{-2}} \left(10^9 \rm~Hz \right)^{-2} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2} \left( b_{\rm max} \right)^{-2} \\\
 & = 0.106 {\rm~Jy} \left( \frac{\nu}{1 \rm~GHz} \right)^{0.7} {\rm~m^2} \left( \frac{\nu}{1 \rm~GHz} \right)^{-2} \left( b_{\rm max} \right)^{-2} \\\
 & = 0.106 {\rm~Jy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-1.3} {\rm~m^2} ({1000 \rm~m})^{-2} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
 & = 1.06 \times 10^{-7} {\rm~Jy} \left( \frac{\nu}{1 \rm~GHz} \right)^{-1.3} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
 & = 1.06 \times 10^{-6} {\rm~Jy} \left( \frac{\nu}{150 \rm~MHz} \right)^{-1.3} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
 & = 3.36 \times 10^{-6} {\rm~Jy} \left( \frac{\nu}{70 \rm~MHz} \right)^{-1.3} \left( \frac{b_{\rm max}}{1000 \rm~m} \right)^{-2} \\\
\end{eqnarray}

I just did this in terms of $\lambda/D$, but should it actually have been in terms of $\lambda / b_{\rm max}$???  I think so... If so, it turns into:
$$
5\sigma = 0.21 {\rm~Jy} \left( \frac{\nu}{70 \rm~MHz} \right)^{-1.3} \left( \frac{D}{1000 \rm~m} \right)^{-2}
$$
dense limit applies to GMRT?, HERA, LOFAR, MWA, SKA, VLA?, LWA

configuration A for VLA to maximize resolution.